In [ ]:
!pip install transformers

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import tensorflow as tf
from tensorflow import keras
from transformers import pipeline, AutoTokenizer, TFAutoModelForCausalLM

tf.keras.mixed_precision.set_global_policy('mixed_float16')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium", padding_side='left')
model = TFAutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at microsoft/DialoGPT-medium.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [ ]:
tokenizer.pad_token = tokenizer.eos_token
text= "Hello dude"
inputs = tokenizer.encode(text + tokenizer.eos_token, return_tensors="tf")

In [ ]:
t_l = ['Hello', 'How are you?']
test_input = tokenizer.encode(t_l, return_tensors="tf")
test_input

<tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[15496,  2437,   389,   345,    30]], dtype=int32)>

In [ ]:
inputs

<tf.Tensor: shape=(1, 3), dtype=int32, numpy=array([[15496, 18396, 50256]], dtype=int32)>

In [ ]:
generate_tokens_ids = model.generate(inputs, pad_token_id=tokenizer.eos_token_id)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
/usr/local/lib/python3.10/dist-packages/transformers/generation/tf_utils.py:834: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length.  recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
generate_tokens_ids[:,:][0]

<tf.Tensor: shape=(6,), dtype=int32, numpy=array([15496, 18396, 50256, 10814,   582, 50256], dtype=int32)>

In [ ]:
tokenizer.decode(generate_tokens_ids[:,inputs.shape[-1]:][0], skip_special_tokens=True)

'Hey man'

In [ ]:
data = pd.read_csv("data.csv")
data.head()

,context_3,context_2,context_1,response
0,NaN,NaN,NaN,For now biggest pornstars are in the west!!‼️
1,NaN,NaN,For now biggest pornstars are in the west!!‼️,NaN
2,NaN,For now biggest pornstars are in the west!!‼️,NaN,How rude!!
3,NaN,NaN,NaN,😂😌❤️
4,NaN,NaN,😂😌❤️,NaN


In [ ]:
data = data.dropna(how="any")
data.head()

,context_3,context_2,context_1,response
11,My Pishi,Yes chick,Hm I've a question in Grammar\nI couldn't find,What is it?
12,Yes chick,Hm I've a question in Grammar\nI couldn't find,What is it?,Which verbs can't be used in present continuou...
23,🤣🤣 this one is good,Are you oky?.,Do you know me sir? 😐🤔,What is the reason for your behavior?.
24,Are you oky?.,Do you know me sir? 😐🤔,What is the reason for your behavior?.,Have you ever observed your behaviour? anbe 😐
28,Hi,Hello,How r u?,Hello


In [ ]:
data.describe()

,context_3,context_2,context_1,response
count,426,426,426,426
unique,389,398,399,392
top,Hi,Hi,Hi,Hi
freq,14,12,10,6


In [ ]:
data = data.replace('\n', ' ', regex=True)
data = data.replace('\s+', ' ', regex=True)
data.head()

,context_3,context_2,context_1,response
11,My Pishi,Yes chick,Hm I've a question in Grammar I couldn't find,What is it?
12,Yes chick,Hm I've a question in Grammar I couldn't find,What is it?,Which verbs can't be used in present continuou...
23,🤣🤣 this one is good,Are you oky?.,Do you know me sir? 😐🤔,What is the reason for your behavior?.
24,Are you oky?.,Do you know me sir? 😐🤔,What is the reason for your behavior?.,Have you ever observed your behaviour? anbe 😐
28,Hi,Hello,How r u?,Hello


In [ ]:
data_ = []

for d in data.iterrows():
    for i in range(len(d[1]) - 1):
        data_.append([d[1][i] + tokenizer.eos_token, d[1][i+1] + tokenizer.eos_token])

In [ ]:
data = pd.DataFrame(data_, columns=['context', 'response'])
print(len(data))
data = data.dropna()
print(len(data))

1278
1278


In [ ]:
enc_dataset = tokenizer(data['context'].tolist(), padding=True, return_tensors="np")
dec_dataset = tokenizer(data['response'].tolist(), padding=True, return_tensors="np")

In [ ]:
train_size = len(enc_dataset['input_ids']) * 80 // 100

In [ ]:
model.compile(
    optimizer=keras.optimizers.AdamW(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy()
)

In [ ]:
tf.random.set_seed(42)

model.fit(
    {
        'input_ids': enc_dataset['input_ids'],
        'attention_mask': enc_dataset['attention_mask'],
        'labels': dec_dataset['input_ids']
    },
    epochs=30,
    callbacks=[keras.callbacks.ModelCheckpoint(filepath="/content/model_dialo.h5", save_best_only=True), keras.callbacks.EarlyStopping(patience=20, monitor="loss")]
)

Epoch 1/30
40/40 [==============================] - ETA: 0s - loss: 6.2302

40/40 [==============================] - 131s 1s/step - loss: 6.2302
Epoch 2/30
40/40 [==============================] - ETA: 0s - loss: 6.2428

40/40 [==============================] - 52s 1s/step - loss: 6.2428
Epoch 3/30
40/40 [==============================] - ETA: 0s - loss: 6.2165

40/40 [==============================] - 53s 1s/step - loss: 6.2165
Epoch 4/30
40/40 [==============================] - ETA: 0s - loss: 6.1857

40/40 [==============================] - 53s 1s/step - loss: 6.1857
Epoch 5/30
40/40 [==============================] - ETA: 0s - loss: 6.1930

...

40/40 [==============================] - 53s 1s/step - loss: 6.1590
Epoch 27/30
40/40 [==============================] - ETA: 0s - loss: 6.1848

40/40 [==============================] - 53s 1s/step - loss: 6.1848
Epoch 28/30
40/40 [==============================] - ETA: 0s - loss: 6.1851

40/40 [==============================] - 53s 1s/step - loss: 6.1851
Epoch 29/30
40/40 [==============================] - ETA: 0s - loss: 6.1585

40/40 [==============================] - 53s 1s/step - loss: 6.1585
Epoch 30/30
40/40 [==============================] - ETA: 0s - loss: 6.1947

40/40 [==============================] - 53s 1s/step - loss: 6.1947


In [ ]:
from huggingface_hub import login

repo_id = "DiTy/dialogpt"
model_name = "DialoGPT"

login(token=TOKEN)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
model.push_to_hub(repo_id)

tf_model.h5:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

In [ ]:
tokenizer.push_to_hub(repo_id)

CommitInfo(commit_url='https://huggingface.co/DiTy/dialogpt/commit/0fd586fe3748b6c07106e3239e13c6bdb6589c83', commit_message='Upload tokenizer', commit_description='', oid='0fd586fe3748b6c07106e3239e13c6bdb6589c83', pr_url=None, pr_revision=None, pr_num=None)